In [1]:
"""
Different ansätze generate different entanglement patterns, affecting trainability and noise sensitivity.

Exercises:
- Visualize both curves and discuss trainability vs. noise.
"""

'\nDifferent ansätze generate different entanglement patterns, affecting trainability and noise sensitivity.\n\nExercises:\n- Visualize both curves and discuss trainability vs. noise.\n'

In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace

def hea_layer(n, theta):
    qc = QuantumCircuit(n)
    for q in range(n): qc.rx(0.8,q); qc.ry(0.5,q); qc.rz(0.3,q)
    for i in range(n-1): qc.rxx(theta,i,i+1)
    return qc

def pair_layer(n, theta):
    qc = QuantumCircuit(n)
    for q in range(n): qc.rx(0.8,q); qc.ry(0.5,q); qc.rz(0.3,q)
    for i in range(0,n-1,2): qc.rxx(theta,i,i+1)
    for i in range(1,n-1,2): qc.rxx(theta,i,i+1)
    return qc

def mid_entropy(psi, n):
    rhoA = partial_trace(DensityMatrix(psi), list(range(n//2))).data # Trace out qubit 1
    evals = np.linalg.eigvalsh(rhoA)
    evals = evals[evals>1e-12]
    return sum(-evals*np.log2(evals))

In [3]:
print("hea_layer for 8 qubits:",)
qc = hea_layer(8,0.2)
print(qc)
print("pair_layer for 8 qubits:")
qc = pair_layer(8,0.2)
print(qc)

hea_layer for 8 qubits:
     ┌─────────┐┌─────────┐┌─────────┐┌───────────┐                          »
q_0: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├┤0          ├──────────────────────────»
     ├─────────┤├─────────┤├─────────┤│  Rxx(0.2) │┌───────────┐             »
q_1: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├┤1          ├┤0          ├─────────────»
     ├─────────┤├─────────┤├─────────┤└───────────┘│  Rxx(0.2) │┌───────────┐»
q_2: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├─────────────┤1          ├┤0          ├»
     ├─────────┤├─────────┤├─────────┤             └───────────┘│  Rxx(0.2) │»
q_3: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├──────────────────────────┤1          ├»
     ├─────────┤├─────────┤├─────────┤                          └───────────┘»
q_4: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├───────────────────────────────────────»
     ├─────────┤├─────────┤├─────────┤                                       »
q_5: ┤ Rx(0.8) ├┤ Ry(0.5) ├┤ Rz(0.3) ├───────────────────────────────────────»
     ├─────────┤├─────────┤├

In [4]:
n=8; L=6; theta=0.4
psi_he = Statevector.from_label("0"*n)
psi_pair = Statevector.from_label("0"*n)
hea, pair, overlap = [], [], []
for d in range(L+1):
    hea.append((d, mid_entropy(psi_he,n)))
    pair.append((d, mid_entropy(psi_pair,n)))
    overlap.append((d, (np.abs(psi_he.data.conj().T @ psi_pair.data))**2))
    if d<L:
        psi_he = psi_he.evolve(hea_layer(n,theta))
        psi_pair = psi_pair.evolve(pair_layer(n,theta))

print("HEA:", hea)
print("Pair:", pair)
print("Overlap:", overlap)

HEA: [(0, np.float64(0.0)), (1, np.float64(0.14109578748254875)), (2, np.float64(0.2368415930101412)), (3, np.float64(0.5144193774363365)), (4, np.float64(0.7858958956013805)), (5, np.float64(0.9265154729319807)), (6, np.float64(1.0996001206224981))]
Pair: [(0, np.float64(0.0)), (1, np.float64(0.14109578748254875)), (2, np.float64(0.23684159301014218)), (3, np.float64(0.5144193774363377)), (4, np.float64(0.7858958956013814)), (5, np.float64(0.9265154729319804)), (6, np.float64(1.0996001206224975))]
Overlap: [(0, np.float64(1.0)), (1, np.float64(0.9999999999999978)), (2, np.float64(0.9999999999999956)), (3, np.float64(0.9999999999999947)), (4, np.float64(0.9999999999999927)), (5, np.float64(0.9999999999999909)), (6, np.float64(0.9999999999999889))]


In [5]:
def hea_layer_alt(n, theta):
    qc = QuantumCircuit(n)
    for q in range(n): qc.rx(0.8,q); qc.ry(0.5,q); qc.rz(0.3,q)
    for i in range(n-1): 
        if i % 2 == 0:
            qc.rxx(theta,i,i+1)
        else:
            qc.ryy(theta,i,i+1)
    return qc

def pair_layer_alt(n, theta):
    qc = QuantumCircuit(n)
    for q in range(n): qc.rx(0.8,q); qc.ry(0.5,q); qc.rz(0.3,q)
    for i in range(0,n-1,2): qc.rxx(theta,i,i+1)
    for i in range(1,n-1,2): qc.ryy(theta,i,i+1)
    return qc

In [6]:
n=8; L=6; theta=0.4
psi_he = Statevector.from_label("0"*n)
psi_pair = Statevector.from_label("0"*n)
hea, pair, overlap = [], [], []
for d in range(L+1):
    hea.append((d, mid_entropy(psi_he,n)))
    pair.append((d, mid_entropy(psi_pair,n)))
    overlap.append((d, (np.abs(psi_he.data.conj().T @ psi_pair.data))**2))
    if d<L:
        psi_he = psi_he.evolve(hea_layer_alt(n,theta))
        psi_pair = psi_pair.evolve(pair_layer_alt(n,theta))

print("HEA:", hea)
print("Pair:", pair)
print("Overlap:", overlap)

HEA: [(0, np.float64(0.0)), (1, np.float64(0.11585759836549395)), (2, np.float64(0.13666544165112313)), (3, np.float64(0.31175719823869114)), (4, np.float64(0.5268886662772366)), (5, np.float64(0.7065218696053106)), (6, np.float64(0.8443450713092262))]
Pair: [(0, np.float64(0.0)), (1, np.float64(0.09273514883335729)), (2, np.float64(0.1651926342187447)), (3, np.float64(0.414436373899551)), (4, np.float64(0.6577144016537176)), (5, np.float64(0.8624055109775808)), (6, np.float64(0.937592949714684))]
Overlap: [(0, np.float64(1.0)), (1, np.float64(0.982439264963681)), (2, np.float64(0.9542857216675236)), (3, np.float64(0.9274834863273084)), (4, np.float64(0.9189695626001178)), (5, np.float64(0.9166720426781583)), (6, np.float64(0.9168803426989058))]
